# Introduction/Business Problem

### This report is for the investors in Chapati On!, a new Indian restaurant (Chapti On!) to be opened in Manchester, England.

### It will assist the investors in deciding the best location for Chapati On!

### There are 128 venues to eat in Manchester, according to Foursquare, and of these, 14 are Indian restaurants.

### This report will provide the location data for these restaurants. It will also provide a recommendation for the best location for a new Indian restaurant in Manchester. 

### The investors can analyse the results to decide where to locate Chapati On!

# Data

## This report utilises postcode data for Manchester, from www.wikipedia.org/wiki/M_postcode_area, latitude and longitude data from www.freemaptools.com/download-uk-postcode-lat-lng.htm, and venue location and ratings data from www.foursquare.com.

## Latitude and longitude data will be merged with Manchester postcodes. This data will then be used to obtain Foursquare data relating to restaurants in the different postcode areas.

## The report will then explore the types of restaurants in the postcode areas with the greatest number of restaurants, and also specifically the greatest number of Indian restaurants.

## The report will then make a recommendation for the best location for Chapati On!

# Methodology

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

!pip install wikipedia
import wikipedia as wp
 


print('Libraries imported.')

Libraries imported.


In [2]:
import urllib.request
#! pip install html5lib

In [3]:
url = 'https://en.wikipedia.org/wiki/M_postcode_area'
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

In [4]:
right_table=soup.find('table', class_='wikitable sortable')

In [5]:
A=[]
B=[]
C=[]
D=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        B.append(cells[0].find(text=True))
        C.append(cells[1].find(text=True))
        
        
for row in right_table.findAll('tr'):
    cells=row.findAll('th')
    if len(cells)==1:
        A.append(cells[0].find(text=True))

In [6]:
df=pd.DataFrame(A,columns=['PostalCode'])
df['PostTown']=B
df['Neighbourhood']=C
df.tail(10)

,PostalCode,PostTown,Neighbourhood
42,M50,SALFORD,Salford Quays
43,M60,MANCHESTER,Large user and PO Box (located geographically ...
44,M60,MANCHESTER,Large user and PO Box (located geographically ...
45,M60,MANCHESTER,Large user and PO Box (located geographically ...
46,M60,MANCHESTER,Large user and PO Box (located geographically ...
47,M60,MANCHESTER,Manchester X
48,M60,SALFORD,Large user and PO Box
49,M61,MANCHESTER,Manchester X
50,M90,MANCHESTER,Manchester Airport
51,M99,MANCHESTER,Manchester X; JD Williams (located geographica...


## Drop rows with PO boxes or other unusable location areas

In [7]:
df=df.drop(df.index[[43,44,45,46,47,48,49,50,51]])
df.tail(10)

,PostalCode,PostTown,Neighbourhood
33,M34,MANCHESTER,Denton
34,M35,MANCHESTER,Failsworth
35,M38,MANCHESTER,Little Hulton
36,M40,MANCHESTER,Collyhurst
37,M41,MANCHESTER,Urmston
38,M43,MANCHESTER,Droylsden
39,M44,MANCHESTER,Irlam
40,M45,MANCHESTER,Whitefield
41,M46,MANCHESTER,Atherton
42,M50,SALFORD,Salford Quays


## Drop duplicate postal code rows

In [8]:
df.drop_duplicates(subset=['PostalCode'], keep='first', inplace=True)
df.shape

(42, 3)

## Drop rows that are not in Manchester

In [9]:
df.PostTown.unique()

array(['MANCHESTER\n', 'SALFORD\n', 'SALE\n'], dtype=object)

In [10]:
df['PostTown'] = df['PostTown'].replace('\n','', regex=True)

In [11]:
df = df[df.PostTown == 'MANCHESTER']

In [12]:
df.shape

(37, 3)

In [13]:
df = df.reset_index(drop=True)
df

,PostalCode,PostTown,Neighbourhood
0,M1,MANCHESTER,Piccadilly
1,M2,MANCHESTER,Deansgate
2,M3,MANCHESTER,City Centre
3,M4,MANCHESTER,Ancoats
4,M8,MANCHESTER,Crumpsall
5,M9,MANCHESTER,Harpurhey
6,M11,MANCHESTER,Clayton
7,M12,MANCHESTER,Ardwick
8,M13,MANCHESTER,Ardwick
9,M14,MANCHESTER,Fallowfield


## Get the latitude and longitude coordinates for neighbourhoods¶

### Download csv file of UK postcodes latitude and longitude

In [14]:
# The code was removed by Watson Studio for sharing.

,id,postcode,latitude,longitude
0,2,AB10,57.13514,-2.11731
1,3,AB11,57.13875,-2.09089
2,4,AB12,57.10100,-2.11060
3,5,AB13,57.10801,-2.23776
4,6,AB14,57.10076,-2.27073


In [15]:
df.PostalCode.unique()

array(['M1\n', 'M2\n', 'M3', 'M4\n', 'M8\n', 'M9\n', 'M11\n', 'M12\n',
       'M13\n', 'M14\n', 'M15\n', 'M16\n', 'M17\n', 'M18\n', 'M19\n',
       'M20\n', 'M21\n', 'M22\n', 'M23\n', 'M24\n', 'M25\n', 'M26\n',
       'M27\n', 'M28\n', 'M29\n', 'M30\n', 'M31\n', 'M32\n', 'M34\n',
       'M35\n', 'M38\n', 'M40\n', 'M41\n', 'M43\n', 'M44\n', 'M45\n',
       'M46\n'], dtype=object)

In [16]:
df['PostalCode'] = df['PostalCode'].replace('\n','', regex=True)

In [17]:
geodf=pd.merge(df,df_data_2, how='left', left_on='PostalCode', right_on='postcode')

In [18]:
geodf

,PostalCode,PostTown,Neighbourhood,id,postcode,latitude,longitude
0,M1,MANCHESTER,Piccadilly,1524,M1,53.47734,-2.23508
1,M2,MANCHESTER,Deansgate,1534,M2,53.48003,-2.24263
2,M3,MANCHESTER,City Centre,1545,M3,53.48361,-2.25018
3,M4,MANCHESTER,Ancoats,1553,M4,53.48455,-2.22909
4,M8,MANCHESTER,Crumpsall,1564,M8,53.50880,-2.23827
5,M9,MANCHESTER,Harpurhey,1565,M9,53.52143,-2.21269
6,M11,MANCHESTER,Clayton,1525,M11,53.47834,-2.17933
7,M12,MANCHESTER,Ardwick,1526,M12,53.46482,-2.20187
8,M13,MANCHESTER,Ardwick,1527,M13,53.46030,-2.21389
9,M14,MANCHESTER,Fallowfield,1528,M14,53.44770,-2.22437


# Analysis

## Explore the post codes

In [19]:
CLIENT_ID = 'I0CF3EV5SN3RWUY0E0OOX1WR4ZE4V4P3ZCJNTPTFGQKIQ2NB'
CLIENT_SECRET = '4EJSOAQ3AR0KLEAAXEL1XC401BOMX1S2MUWEFJMSBM1LCDWV' 
VERSION = '20180605' 
radius = 500
LIMIT = 100

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCodeLatitude', 
                  'PostalCodeLongitude', 
                  'Venue', 
                  'VenueLatitude', 
                  'VenueLongitude', 
                  'VenueCategory']
    
    return(nearby_venues)

In [21]:
manchester_venues = getNearbyVenues(names=geodf['PostalCode'],
                                   latitudes=geodf['latitude'],
                                   longitudes=geodf['longitude']
                                  )

M1
M2
M3
M4
M8
M9
M11
M12
M13
M14
M15
M16
M17
M18
M19
M20
M21
M22
M23
M24
M25
M26
M27
M28
M29
M30
M31
M32
M34
M35
M38
M40
M41
M43
M44
M45
M46


In [22]:
print(manchester_venues.shape)
manchester_venues.head()

(515, 7)


,PostalCode,PostalCodeLatitude,PostalCodeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,M1,53.47734,-2.23508,The Molly House,53.477325,-2.237201,Pub
1,M1,53.47734,-2.23508,Richmond Tea Rooms,53.477652,-2.236810,Tea Room
2,M1,53.47734,-2.23508,DoubleTree by Hilton,53.478663,-2.233560,Hotel
3,M1,53.47734,-2.23508,Habesha,53.477220,-2.237309,Ethiopian Restaurant
4,M1,53.47734,-2.23508,Piccadilly Place,53.477720,-2.233302,Plaza


In [23]:
manchester_venues.groupby('PostalCode').count()

,PostalCodeLatitude,PostalCodeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,,,,,,
M1,100,100,100,100,100,100
M11,2,2,2,2,2,2
M13,4,4,4,4,4,4
M14,9,9,9,9,9,9
M15,16,16,16,16,16,16
M16,4,4,4,4,4,4
M17,2,2,2,2,2,2
M18,7,7,7,7,7,7
M19,4,4,4,4,4,4


In [24]:
manchester_venues.VenueCategory.unique()

array(['Pub', 'Tea Room', 'Hotel', 'Ethiopian Restaurant', 'Plaza',
       'Monument / Landmark', 'Gay Bar', 'Adult Boutique', 'Park',
       'Indian Restaurant', 'Restaurant', 'Fast Food Restaurant',
       'Japanese Restaurant', 'Coffee Shop', 'Gym', 'Thai Restaurant',
       'Ramen Restaurant', 'Beer Bar', 'Bar', 'Steakhouse',
       'Falafel Restaurant', 'Cocktail Bar', 'Chinese Restaurant',
       'Music Store', 'Art Gallery', 'Sandwich Place', 'Grocery Store',
       'Sushi Restaurant', 'Bakery', 'Café', 'Theater', 'Bubble Tea Shop',
       'Hobby Shop', 'Chocolate Shop', 'Pizza Place',
       'Lebanese Restaurant', 'Portuguese Restaurant',
       'Modern European Restaurant', 'Fish & Chips Shop',
       'Vietnamese Restaurant', 'Donut Shop', 'Juice Bar', 'Pool',
       'Train Station', 'Sculpture Garden', 'Italian Restaurant',
       'Casino', 'Asian Restaurant', 'Wine Bar', 'Tapas Restaurant',
       'Bike Shop', 'BBQ Joint', 'Bookstore', 'Gym / Fitness Center',
       'Burger 

In [25]:
restaurants=manchester_venues.copy()

In [26]:
restaurants= restaurants[restaurants['VenueCategory'].str.contains('Restaurant|Steakhouse|Pizza|Fish|Joint|Burrito|Truck|Bistro')]
restaurants = restaurants.reset_index(drop=True)
restaurants

,PostalCode,PostalCodeLatitude,PostalCodeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,M1,53.47734,-2.23508,Habesha,53.477220,-2.237309,Ethiopian Restaurant
1,M1,53.47734,-2.23508,Bundobust,53.480870,-2.234693,Indian Restaurant
2,M1,53.47734,-2.23508,No 1 Canal Street,53.478064,-2.235147,Restaurant
3,M1,53.47734,-2.23508,Northern Soul,53.478732,-2.232473,Fast Food Restaurant
4,M1,53.47734,-2.23508,Yuzu,53.478899,-2.239170,Japanese Restaurant
5,M1,53.47734,-2.23508,Try Thai,53.478072,-2.240284,Thai Restaurant
6,M1,53.47734,-2.23508,Shoryu Ramen,53.480774,-2.236363,Ramen Restaurant
7,M1,53.47734,-2.23508,The Grill,53.479761,-2.238536,Steakhouse
8,M1,53.47734,-2.23508,Go Falafel,53.481057,-2.234222,Falafel Restaurant
9,M1,53.47734,-2.23508,Hunan,53.478968,-2.240150,Chinese Restaurant


In [27]:
restaurants.shape

(128, 7)

In [28]:
indianrestaurants= restaurants[restaurants['VenueCategory'].str.contains('Indian')]
indianrestaurants.shape

(14, 7)

### Foursquare has details of 128 restaurants in Manchester.
### Of these, 14 are Indian restaurants.

## Map of restaurants

In [29]:
map = folium.Map(location=[53.483959, -2.244644], zoom_start=12, width='100%', height='100%')

for lat, lng, label in zip(restaurants.VenueLatitude, restaurants.VenueLongitude, restaurants.VenueCategory):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map)
    
for lat, lng, label in zip(indianrestaurants.VenueLatitude, indianrestaurants.VenueLongitude, indianrestaurants.VenueCategory):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        fill=True,
        popup=label,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map)
    
map

In [30]:
restaurantscount=restaurants.groupby('PostalCode').count()
restaurantscount

,PostalCodeLatitude,PostalCodeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,,,,,,
M1,29,29,29,29,29,29
M11,1,1,1,1,1,1
M13,1,1,1,1,1,1
M14,1,1,1,1,1,1
M15,2,2,2,2,2,2
M19,2,2,2,2,2,2
M2,32,32,32,32,32,32
M20,3,3,3,3,3,3
M21,5,5,5,5,5,5


## There are 4 postal codes with a high density of restaurants. Let's explore what types of restaurants there are in these 4 postal codes

In [31]:
top4=restaurants.groupby('PostalCode').count().nlargest(4, 'VenueCategory')

In [32]:
top4

,PostalCodeLatitude,PostalCodeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,,,,,,
M2,32,32,32,32,32,32
M1,29,29,29,29,29,29
M3,22,22,22,22,22,22
M4,10,10,10,10,10,10


In [33]:
top4 = restaurants[restaurants['PostalCode'].isin(['M2','M1','M3','M4'])]

In [34]:
top4

,PostalCode,PostalCodeLatitude,PostalCodeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,M1,53.47734,-2.23508,Habesha,53.477220,-2.237309,Ethiopian Restaurant
1,M1,53.47734,-2.23508,Bundobust,53.480870,-2.234693,Indian Restaurant
2,M1,53.47734,-2.23508,No 1 Canal Street,53.478064,-2.235147,Restaurant
3,M1,53.47734,-2.23508,Northern Soul,53.478732,-2.232473,Fast Food Restaurant
4,M1,53.47734,-2.23508,Yuzu,53.478899,-2.239170,Japanese Restaurant
5,M1,53.47734,-2.23508,Try Thai,53.478072,-2.240284,Thai Restaurant
6,M1,53.47734,-2.23508,Shoryu Ramen,53.480774,-2.236363,Ramen Restaurant
7,M1,53.47734,-2.23508,The Grill,53.479761,-2.238536,Steakhouse
8,M1,53.47734,-2.23508,Go Falafel,53.481057,-2.234222,Falafel Restaurant
9,M1,53.47734,-2.23508,Hunan,53.478968,-2.240150,Chinese Restaurant


In [35]:
top4.shape

(93, 7)

## Map of postal codes M1, M2, M3 and M4

In [36]:
top4indian= top4[top4['VenueCategory'].str.contains('Indian')]
top4indian.shape

(8, 7)

In [37]:
maptop4 = folium.Map(location=[53.483959, -2.244644], zoom_start=15, width='100%', height='100%')

for lat, lng, label in zip(top4.VenueLatitude, top4.VenueLongitude, top4.VenueCategory):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(maptop4)
    
for lat, lng, label in zip(top4indian.VenueLatitude, top4indian.VenueLongitude, top4indian.VenueCategory):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        fill=True,
        popup=label,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(maptop4)
    
maptop4

## There are no Indian restaurants in several areas which have other types of restaurants nearby.

## These are the Northern Quarter, Spinningfields, and Manchester Piccadilly

In [38]:
recommendations=pd.DataFrame(np.array([['Northern Quarter', 53.48429,-2.236283],['Spinningfields', 53.479523,-2.249866],['Manchester Piccadilly', 53.477847,-2.231893]]),columns=['Area','Latitude','Longitude'])

recommendationsmap = folium.Map(location=[53.483959, -2.244644], zoom_start=15, width='100%', height='100%')

for lat, lng, label in zip(top4.VenueLatitude, top4.VenueLongitude, top4.VenueCategory):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(recommendationsmap)
    
for lat, lng, label in zip(top4indian.VenueLatitude, top4indian.VenueLongitude, top4indian.VenueCategory):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        fill=True,
        popup=label,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(recommendationsmap)
    
for lat, lng, label in zip(recommendations.Latitude, recommendations.Longitude, recommendations.Area):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        color='gree',
        fill=True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(recommendationsmap)   

from folium.features import DivIcon

folium.map.Marker(
    [53.48429,-2.236283],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 14pt">Northern Quarter</div>',
        )
    ).add_to(recommendationsmap)

folium.map.Marker(
    [53.479523,-2.249866],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 14pt">Spinningfields</div>',
        )
    ).add_to(recommendationsmap)

folium.map.Marker(
    [53.477847,-2.231893],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 14pt">Piccadilly</div>',
        )
    ).add_to(recommendationsmap)
   
recommendationsmap

## Explore other venues near these three locations

In [39]:
top3= getNearbyVenues(names=recommendations['Area'],
                                   latitudes=recommendations['Latitude'],
                                   longitudes=recommendations['Longitude']
                                  )

Northern Quarter
Spinningfields
Manchester Piccadilly


In [40]:
top3

,PostalCode,PostalCodeLatitude,PostalCodeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,Northern Quarter,53.48429,-2.236283,Piccadilly Records,53.482801,-2.235603,Record Shop
1,Northern Quarter,53.48429,-2.236283,Federal,53.484234,-2.238594,Café
2,Northern Quarter,53.48429,-2.236283,Mackie Mayor Building,53.485488,-2.234870,Food Court
3,Northern Quarter,53.48429,-2.236283,Pieminister,53.482834,-2.236190,Pie Shop
4,Northern Quarter,53.48429,-2.236283,Fig and Sparrow,53.482626,-2.235609,Coffee Shop
5,Northern Quarter,53.48429,-2.236283,Northern Soul • Grilled Cheese,53.483074,-2.238303,Sandwich Place
6,Northern Quarter,53.48429,-2.236283,Just Between Friends,53.483454,-2.235236,Coffee Shop
7,Northern Quarter,53.48429,-2.236283,Siop,53.484374,-2.233957,Donut Shop
8,Northern Quarter,53.48429,-2.236283,Sugar Junction,53.483545,-2.235103,Tea Room
9,Northern Quarter,53.48429,-2.236283,Band on the Wall,53.485290,-2.234041,Concert Hall


In [41]:
top3.shape

(300, 7)

In [42]:
top3other= top3[~top3['VenueCategory'].str.contains('Restaurant|Steakhouse|Pizza|Fish|Joint|Burrito|Truck|Bistro|Tea|Coffee|Sandwich|Bakery|Café|Bagel|Food|Cafeteria|Deli')]
top3other = top3other.reset_index(drop=True)
top3other

,PostalCode,PostalCodeLatitude,PostalCodeLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,Northern Quarter,53.48429,-2.236283,Piccadilly Records,53.482801,-2.235603,Record Shop
1,Northern Quarter,53.48429,-2.236283,Pieminister,53.482834,-2.236190,Pie Shop
2,Northern Quarter,53.48429,-2.236283,Siop,53.484374,-2.233957,Donut Shop
3,Northern Quarter,53.48429,-2.236283,Band on the Wall,53.485290,-2.234041,Concert Hall
4,Northern Quarter,53.48429,-2.236283,Vinyl Exchange,53.482581,-2.235641,Record Shop
5,Northern Quarter,53.48429,-2.236283,Matt & Phreds Jazz Club,53.483434,-2.234707,Jazz Club
6,Northern Quarter,53.48429,-2.236283,Smithfield Market Tavern,53.485466,-2.234404,Pub
7,Northern Quarter,53.48429,-2.236283,Washhouse,53.484642,-2.239473,Speakeasy
8,Northern Quarter,53.48429,-2.236283,57 Thomas Street Marble Bar,53.483844,-2.236292,Bar
9,Northern Quarter,53.48429,-2.236283,Fred Aldous,53.482530,-2.234045,Arts & Crafts Store


In [43]:
top3other_onehot = pd.get_dummies(top3other[['VenueCategory']], prefix="", prefix_sep="")
top3other_onehot['PostalCode'] = top3other['PostalCode'] 

fixed_columns = [top3other_onehot.columns[-1]] + list(top3other_onehot.columns[:-1])
top3other_onehot = top3other_onehot[fixed_columns]

top3other_onehot.head()

,PostalCode,Adult Boutique,Arts & Crafts Store,Bar,Beer Bar,Beer Store,Bike Shop,Bookstore,Brewery,Butcher,Casino,Chocolate Shop,Clothing Store,Cocktail Bar,Comic Shop,Concert Hall,Convenience Store,Department Store,Dessert Shop,Dive Bar,Donut Shop,Electronics Store,Escape Room,Furniture / Home Store,Gay Bar,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hobby Shop,Hotel,Hotel Bar,Jazz Club,Jewelry Store,Juice Bar,Market,Monument / Landmark,Movie Theater,Museum,Music Store,Outdoor Supply Store,Outlet Store,Park,Pharmacy,Pie Shop,Piercing Parlor,Plaza,Pub,Record Shop,Residential Building (Apartment / Condo),Science Museum,Sculpture Garden,Shopping Mall,Snack Place,Speakeasy,Sporting Goods Shop,Theater,Toy / Game Store,Train Station,Whisky Bar,Women's Store
0,Northern Quarter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Northern Quarter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Northern Quarter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Northern Quarter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Northern Quarter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
top3other_grouped = top3other_onehot.groupby('PostalCode').mean().reset_index()

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
top3other_venues_sorted = pd.DataFrame(columns=columns)
top3other_venues_sorted['PostalCode'] = top3other_grouped['PostalCode']

for ind in np.arange(top3other_grouped.shape[0]):
    top3other_venues_sorted.iloc[ind, 1:] = return_most_common_venues(top3other_grouped.iloc[ind, :], num_top_venues)

top3other_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manchester Piccadilly,Gay Bar,Hotel,Bar,Beer Bar,Pub,Cocktail Bar,Train Station,Plaza,Monument / Landmark,Juice Bar
1,Northern Quarter,Bar,Pub,Bookstore,Record Shop,Arts & Crafts Store,Toy / Game Store,Beer Bar,Cocktail Bar,Plaza,Convenience Store
2,Spinningfields,Hotel,Bar,Plaza,Pub,Grocery Store,Cocktail Bar,Gym,Science Museum,Bookstore,Casino


# Conclusion

## Analysing the types of other venues in each of the different areas, Manchester Piccadilly has the most drinking establishments nearby compared to the other two areas. It is also close to the train station.

## The recommendation to the investors of Chapati On! is to locate the restaurant in the Manchester Piccadilly area.